In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [4]:
item = msm.convert('pdb_id:1l2y', to_form='mmtf.MMTFDecoder')

In [5]:
molsys = msm.convert('pdb_id:1l2y')

NameError: name 'get_bonded_atom_pairs' is not defined

In [ ]:
molsys.topology.bonds

In [ ]:
38*20

In [ ]:
molsys.topology.groups[:30]

In [ ]:
coordinates = np.column_stack([item.x_coord_list, item.y_coord_list, item.z_coord_list])

In [ ]:
coordinates = coordinates.reshape([-1, item.num_atoms, 3])

In [ ]:
coordinates.shape

In [ ]:
item.num_atoms

In [ ]:
item.num_models*304

In [ ]:
len(item.sec_struct_list)

In [ ]:
coordinates = np.column_stack([item.x_coord_list, item.y_coord_list, item.z_coord_list])
coordinates = coordinates.reshape([-1, item.num_atoms, 3])
coordinates = puw.quantity(coordinates, 'angstroms')
coordinates = puw.standardize(coordinates)

In [ ]:
38*20

In [ ]:
item.unit_cell

In [ ]:
import numpy as np

In [ ]:
aa = np.array(item.alt_loc_list)